In [29]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data//reset") if os.path.isfile(os.path.join("data//reset", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"data//reset//"+p, data_list))[:1] #
print(file_path)

['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240811_2240.raw']


In [15]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(VTools)
import matplotlib.pyplot as plt

['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240811_2240.raw'] # only 0,1 channels
#file_path = paths = ['C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240731_0217.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0040.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240810_0840.raw']


file = file_path[0]
file_name = file.split("//")[-1][:-4]
print(file_name)
fReader = rawFileReader.fileReader(file)
max_chunks = 10

proccesedChunk = 0
bcr_time = 0
trigger_count = 0
bcr_count = 0
time_window = 100 #in seconds
triggers_rate = []
dates = []
channels = (1,2) #(1,0)
tdc5Reads = []
#maybe later against time
time = []
with tqdm(total=max_chunks, desc=f"Processing {file_name}", unit='Chunks') as pbar:
    while proccesedChunk < max_chunks:
        initial_time = None
        trigger_count = 0
        bcr_count = 0        
        time_collected = 0
        all_channels = set()
        while time_collected < time_window:
            if not fReader.readBlock():
                print("Finished Reading File")
                max_chunks = event_count
                break

            if(fReader.hasEvents()):
                tdc5Reads = fReader.getTDCFiveEvents()
                if not tdc5Reads:
                    continue
                if not initial_time:
                    initial_time = tdc5Reads[0][0]
                    print("Initial Time:", initial_time)
                for word in tdc5Reads[0][1]:
                    tdcChannel = (word>>24)&0x7f
                    tdcHitTime = word&0xfffff
                    """
                    if tdcChannel == channels[1]:
                        bcr_count += 1
                        check = tdcHitTime-bcr_time
                        if check:
                            bcr_time = tdcHitTime
                            print("--------------------")
                            print("BCR Update",bcr_time)
                            print("Check:", check)
                    """
                    if tdcChannel == channels[0]:
                        trigger_count += 1
                    time_collected = (tdc5Reads[0][0]-initial_time).total_seconds()
                        
                
        #print("BCR Count:", br_count)
        #print("Trigger Count:", triggers_count)
        event_count = event_count+1
        if initial_time:
            time_diff = (tdc5Reads[0][0]-initial_time).total_seconds()
        if tdc5Reads and time_diff:
            dates.append(tdc5Reads[0][0])
            triggers_rate.append(trigger_count/(time_diff))
            proccesedChunk += 1
            pbar.update(1)
            

plt.plot_date(dates, triggers_rate, xdate=True, linestyle='solid', label=f"{file_name}")
plt.xlabel("Event Chunks")
plt.ylim(0, 8000)
plt.ylabel("Triggers Rate (Hz)")
plt.title(f"Triggers Rate vs Event Chunks")
plt.show()
                    
                 
        
    

proAnubis_240731_0017


Processing proAnubis_240731_0017:   0%|          | 0/5 [00:00<?, ?Chunks/s]

Initial Time: 2024-07-30 23:17:53.026146


Processing proAnubis_240731_0017:  20%|██        | 1/5 [00:20<01:20, 20.12s/Chunks]

Initial Time: 2024-07-30 23:19:33.029881


...

Processing proAnubis_240731_0017:  60%|██████    | 3/5 [01:04<00:42, 21.42s/Chunks]

Initial Time: 2024-07-30 23:22:53.036336


Processing proAnubis_240731_0017:  80%|████████  | 4/5 [01:31<00:23, 23.61s/Chunks]

Initial Time: 2024-07-30 23:24:33.039684


Processing proAnubis_240731_0017: 100%|██████████| 5/5 [01:54<00:00, 23.00s/Chunks]
C:\Users\jony\AppData\Local\Temp\ipykernel_23068\2276687316.py:78: MatplotlibDeprecationWarning: The plot_date function was deprecated in Matplotlib 3.9 and will be removed in 3.11. Use plot instead.
  plt.plot_date(dates, triggers_rate, xdate=True, linestyle='solid', label=f"{file_name}")


In [24]:
plt.plot(dates, triggers_rate, linestyle='solid', label=f"{file_name}")
plt.xlabel("Date")
plt.ylim(0, 8000)
plt.ylabel("Triggers Rate (Hz)")
plt.xticks(dates, [d.strftime("%H:%M:%S") for d in dates])
plt.title(f"Triggers Rate")
plt.show()